In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
new_var = sys.path.insert(0, '/content/gdrive/My Drive/Colab Notebooks/libraries')
new_var

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA
from langchain_community.document_loaders import WebBaseLoader



: 

In [21]:
loader=WebBaseLoader('https://www.csueastbay.edu/msba/')
documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)


In [22]:
len(final_documents)

7

In [23]:
import sys
if '/content/gdrive/My Drive/Colab Notebooks/libraries' in sys.path:
    sys.path.remove('/content/gdrive/My Drive/Colab Notebooks/libraries')
import torch

# Check if a GPU is available and if it's a T4, otherwise use CPU
if torch.cuda.is_available() and "T4" in torch.cuda.get_device_name(0):
    device = 'cuda'
    print("T4 GPU is available and will be used.")
else:
    device = 'cpu'
    print("T4 GPU is not available, using CPU.")

T4 GPU is available and will be used.


In [6]:
device

'cuda'

In [24]:
sys.path.insert(0, '/content/gdrive/My Drive/Colab Notebooks/libraries')
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",    # You can also use "sentence-transformers/all-MiniLM-l6-v2" if needed
    model_kwargs={'device': device},        # Automatically use GPU if available
    encode_kwargs={'normalize_embeddings': True}
)

# Example usage: embedding a sentence
sentences = ["This is an example sentence.", "This is another sentence."]
embeddings = huggingface_embeddings.embed_documents(sentences)
print(embeddings)

[[-0.032561976462602615, 0.006154800299555063, 0.02186093106865883, -0.03118247538805008, -0.01648002117872238, -0.029896792024374008, 0.03834739327430725, 0.03136291354894638, 0.05026160553097725, -0.03578374162316322, -0.015108088962733746, 0.0031268917955458164, -0.006939791142940521, -0.004551147576421499, 0.03473673015832901, 0.025775453075766563, -0.00027096670237369835, -0.011834933422505856, -0.058547746390104294, 0.012845851480960846, 0.0715881735086441, -0.002421964192762971, -0.006292664911597967, 0.025896189734339714, -0.023338623344898224, 0.004941392224282026, 0.006943656597286463, -0.01722186990082264, 0.03235581889748573, -0.05418558046221733, -0.02201775275170803, -0.002187266480177641, -0.006508809980005026, 0.016384240239858627, 0.010943491011857986, 0.039968617260456085, -0.07783515006303787, 0.024936404079198837, -0.09552782028913498, 0.012408279813826084, 0.03750612959265709, 0.037886254489421844, -0.006826899480074644, 0.03875800967216492, -0.014658131636679173, 

In [25]:
vectorstore=FAISS.from_documents(final_documents,huggingface_embeddings)

save_path = "/content/gdrive/My Drive/Colab Notebooks/vectorstore"

# Save the FAISS vector store
vectorstore.save_local(save_path)

In [28]:
vectordb = FAISS.load_local(save_path, huggingface_embeddings, allow_dangerous_deserialization=True)

query="number of courses"
relevant_docments=vectordb.similarity_search(query)

print(relevant_docments[0].page_content)

Request Information












                                                                            PROGRAM HIGHLIGHTS
                                                                    




                                                                            PROGRAM OBJECTIVES
                                                                    




                                                                            WHO SHOULD ATTEND
                                                                    





Type of degree conferred: Master of Science
Number of courses: 10-12
Program length: flexible, 2 semesters - 6 semesters
Strong alumni network
A STEM program: 36-month OPT for international students
Flexibility: Progress at your own pace with evening/weekend/online/hybrid classes
Experiential hands-on learning


In [29]:
#retriever=vectordb.as_retriever(search_type="similarity",search_kwargs={"k":3})
retriever = vectordb.as_retriever(score_threshold = 0.7)
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x781428820e50> search_kwargs={}


In [30]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)


In [31]:
from langchain_core.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}




In [32]:
chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [37]:
chain('who is program coordinator')

{'query': 'who is program coordinator',
 'result': 'The Program Coordinator for the Master of Business Analytics is Dr. Surendra Sarnikar.',
 'source_documents': [Document(metadata={'source': 'https://www.csueastbay.edu/msba/', 'title': 'Master of Science in Business Analytics- College of Business & Economics | CSUEB', 'description': 'The STEM-designated MS in Business Analytics program at Cal State East Bay focuses on statistics, mathematics, business intelligence, and big data analytics.', 'language': 'en'}, page_content='Request Information\n\n\n\n\n\n\n\n\n\n\n\n\n                                                                            PROGRAM HIGHLIGHTS\n                                                                    \n\n\n\n\n                                                                            PROGRAM OBJECTIVES\n                                                                    \n\n\n\n\n                                                                            W